In [1]:
import nba_py.league as nba
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import datetime as dt

class NBAstats(object ):
    def __init__(self, roster_path='/Users/schirmer/Documents/Personal stuff/lfbl/rosters_2017-18.xlsx'):
        """This class gets NBA stats for LFBL fantasy"""
        self.roster_path=roster_path
        self.set_rosters(roster_path)
        self.set_fant_params()
        self.set_game_log()
        self.set_player_info()

    def __strip_ast(self,name):
        if name[-1]=='*':
            return name[:-1]
        else:
            return name      

    def set_fant_params(self):
        self.std_scaler=StandardScaler()
        self.lin_reg=LinearRegression()

        self.stats17=nba.PlayerStats(season='2017-18').overall().set_index('PLAYER_NAME')
        fantasy_pts=self.stats17.NBA_FANTASY_PTS.values.ravel()
        x_vals=self.std_scaler.fit_transform(self.stats17.loc[:,['PTS','FG3M','REB','AST','STL','BLK','TOV']])
        self.lin_reg.fit(x_vals,fantasy_pts)       
        
    def set_game_log(self):
        gamelog=nba.GameLog()
        self.all_games=gamelog.overall()
        self.all_games['GAME_DATE']=pd.to_datetime(self.all_games['GAME_DATE'])
        self.all_games=pd.merge(self.all_games, self.rosters, how='outer')
        self.all_games['Team'].fillna('FA', inplace=True)
        self.all_games=self.all_games[self.all_games.GAME_DATE.notnull()]
        x_vals=self.std_scaler.transform(self.all_games.loc[:,['PTS','FG3M','REB','AST','STL','BLK','TOV']])
        self.all_games['FANT']=self.lin_reg.predict(x_vals)
        self.all_games=pd.merge(self.all_games, self.stats17.loc[:,['NBA_FANTASY_PTS']], left_on='PLAYER_NAME',right_index=True, how='outer')
    
    def get_game_log(self, name=''):
        return self.all_games[self.all_games.PLAYER_NAME.apply(lambda x: x.find(name)!=-1)].sort_values(by='GAME_DATE', ascending=0)
    
    def set_player_info(self):
        self.all_players=pd.DataFrame(self.all_games['PLAYER_NAME'].unique(),columns=['PLAYER_NAME'])
        self.all_players['Team']='None'
        self.all_players=pd.merge(self.all_players, self.rosters, how='outer')
        
    def get_player_info(self):
        return self.all_players
    
    def set_rosters(self,roster_path):
        self.rosters=pd.read_excel(roster_path,"Sheet2",parse_cols='D:E')
        self.rosters.dropna(inplace=True)
        self.rosters.PLAYER_NAME=self.rosters.PLAYER_NAME.apply(lambda x: self.__strip_ast(x))
        
    def get_rosters(self):
        return self.rosters
        
    def daily(self, date=(dt.datetime.today()-dt.timedelta(days=1)).date(), sort_stat='FANT', team='THUNDERING CORNBREAD'):
        if sort_stat != 'FANT':
            sort_stat=[sort_stat, 'FANT']
        
        return self.all_games[(self.all_games['GAME_DATE'] >=date) & 
                              (self.all_games['Team']==team)].loc[:,['PLAYER_NAME','TEAM_ABBREVIATION','GAME_DATE', 'MIN', 
                                            'FG3M','REB','AST','STL','BLK','TOV',
                                            'PTS','FANT','NBA_FANTASY_PTS','Team']].sort_values(by=sort_stat, ascending=False)
        

In [2]:
nbastats=NBAstats()

In [3]:
nbastats.daily(team='FA')

,PLAYER_NAME,TEAM_ABBREVIATION,GAME_DATE,MIN,FG3M,REB,AST,STL,BLK,TOV,PTS,FANT,NBA_FANTASY_PTS,Team
656,Donovan Mitchell,UTA,2017-11-13,35.0,3.0,4.0,4.0,4.0,1.0,2.0,24.0,47.698948,24.8,FA
1218,Kent Bazemore,ATL,2017-11-13,30.0,3.0,4.0,7.0,3.0,0.0,1.0,22.0,45.153765,29.9,FA
448,Tyreke Evans,MEM,2017-11-13,33.0,3.0,6.0,3.0,0.0,0.0,1.0,27.0,37.648522,30.8,FA
2261,John Henson,MIL,2017-11-13,25.0,0.0,8.0,3.0,0.0,2.0,0.0,17.0,36.883407,21.6,FA
826,Jordan Clarkson,LAL,2017-11-13,26.0,3.0,3.0,3.0,1.0,0.0,0.0,25.0,35.982881,22.8,FA
865,Dwyane Wade,CLE,2017-11-13,28.0,0.0,8.0,3.0,2.0,1.0,3.0,15.0,35.050717,21.2,FA
978,E'Twaun Moore,NOP,2017-11-13,34.0,2.0,2.0,4.0,1.0,0.0,2.0,24.0,33.327092,21.5,FA
1165,Marco Belinelli,ATL,2017-11-13,27.0,4.0,3.0,3.0,4.0,0.0,2.0,14.0,32.144904,21.0,FA
1111,Kyle Korver,CLE,2017-11-13,29.0,5.0,5.0,1.0,1.0,0.0,0.0,21.0,31.530000,16.6,FA
2473,Taj Gibson,MIN,2017-11-13,39.0,1.0,10.0,0.0,1.0,1.0,2.0,15.0,31.017655,25.2,FA


In [64]:
totals(res,'2017-10-20','2017-10-31')

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Team,FANT,NBA_FANTASY_PTS
0,22017,203507.0,Giannis Antetokounmpo,1.610613e+09,MIL,Milwaukee Bucks,0021700034,2017-10-21,MIL vs. POR,W,...,2.0,2.0,2.0,3.0,44.0,3.0,1.0,KING'S COURT,69.531125,57.3
2,22017,203507.0,Giannis Antetokounmpo,1.610613e+09,MIL,Milwaukee Bucks,0021700021,2017-10-20,MIL vs. CLE,L,...,3.0,1.0,2.0,2.0,34.0,-17.0,1.0,KING'S COURT,65.513066,57.3
3,22017,203507.0,Giannis Antetokounmpo,1.610613e+09,MIL,Milwaukee Bucks,0021700085,2017-10-29,MIL @ ATL,W,...,0.0,0.0,1.0,4.0,33.0,8.0,1.0,KING'S COURT,54.146149,57.3
4,22017,203507.0,Giannis Antetokounmpo,1.610613e+09,MIL,Milwaukee Bucks,0021700044,2017-10-23,MIL vs. CHA,W,...,1.0,2.0,6.0,5.0,32.0,7.0,1.0,KING'S COURT,60.767814,57.3
5,22017,203507.0,Giannis Antetokounmpo,1.610613e+09,MIL,Milwaukee Bucks,0021700103,2017-10-31,MIL vs. OKC,L,...,0.0,1.0,4.0,4.0,28.0,-15.0,2.0,KING'S COURT,41.100966,57.3
6,22017,203507.0,Giannis Antetokounmpo,1.610613e+09,MIL,Milwaukee Bucks,0021700067,2017-10-26,MIL vs. BOS,L,...,3.0,1.0,3.0,3.0,28.0,-10.0,2.0,KING'S COURT,59.406149,57.3
9,22017,203114.0,Khris Middleton,1.610613e+09,MIL,Milwaukee Bucks,0021700085,2017-10-29,MIL @ ATL,W,...,2.0,0.0,2.0,2.0,27.0,26.0,1.0,YOSEMITE 2ND PLACE,52.828679,34.5
10,22017,203114.0,Khris Middleton,1.610613e+09,MIL,Milwaukee Bucks,0021700044,2017-10-23,MIL vs. CHA,W,...,1.0,1.0,2.0,3.0,20.0,10.0,1.0,YOSEMITE 2ND PLACE,35.660486,34.5
11,22017,203114.0,Khris Middleton,1.610613e+09,MIL,Milwaukee Bucks,0021700034,2017-10-21,MIL vs. POR,W,...,0.0,0.0,2.0,5.0,18.0,1.0,1.0,YOSEMITE 2ND PLACE,28.890579,34.5
12,22017,203114.0,Khris Middleton,1.610613e+09,MIL,Milwaukee Bucks,0021700067,2017-10-26,MIL vs. BOS,L,...,2.0,0.0,3.0,4.0,15.0,-3.0,2.0,YOSEMITE 2ND PLACE,23.399660,34.5


In [65]:
z=res.groupby('Team').apply(totals, '2017-10-20','2017-10-31')

In [66]:
y=z[['FG3M','REB','AST','STL','BLK','TOV','PTS']].groupby('Team').sum()

In [67]:
for name,games in z['GAME_ID'].groupby('Team'):
    print(name )
    print(y.loc[name]/games.count())

BAFKETBALL JONES
FG3M     1.396825
REB      5.888889
AST      2.682540
STL      0.952381
BLK      0.730159
TOV      1.952381
PTS     14.825397
Name: BAFKETBALL JONES, dtype: float64
COOK N CHILL
FG3M     1.410959
REB      5.917808
AST      3.054795
STL      0.945205
BLK      0.767123
TOV      2.438356
PTS     16.479452
Name: COOK N CHILL, dtype: float64
DUDES W/ DUDES
FG3M     0.941176
REB      5.279412
AST      3.691176
STL      1.147059
BLK      0.352941
TOV      2.367647
PTS     13.088235
Name: DUDES W/ DUDES, dtype: float64
JUGHEAD'S STINKPALM
FG3M     1.101266
REB      6.810127
AST      3.164557
STL      1.101266
BLK      0.759494
TOV      2.354430
PTS     15.012658
Name: JUGHEAD'S STINKPALM, dtype: float64
KAPTAIN'S LOG
FG3M     1.728395
REB      5.358025
AST      3.765432
STL      1.222222
BLK      0.469136
TOV      2.456790
PTS     18.111111
Name: KAPTAIN'S LOG, dtype: float64
KING'S COURT
FG3M     1.341772
REB      6.177215
AST      3.569620
STL      1.025316
BLK      0.594937

In [162]:
nbastats.daily(team='FA')

,PLAYER_NAME,TEAM_ABBREVIATION,GAME_DATE,MIN,FG3M,REB,AST,STL,BLK,TOV,PTS,FANT,NBA_FANTASY_PTS,Team
221,JJ Redick,PHI,2017-11-03,33.0,8.0,3.0,6.0,0.0,0.0,1.0,31.0,42.503708,23.6,FA
1892,Jerian Grant,CHI,2017-11-04,36.0,1.0,8.0,9.0,1.0,1.0,0.0,13.0,41.910015,26.6,FA
1900,Brandan Wright,MEM,2017-11-04,25.0,0.0,9.0,2.0,1.0,4.0,0.0,13.0,41.593906,14.8,FA
569,Justin Holiday,CHI,2017-11-04,43.0,3.0,9.0,1.0,3.0,1.0,1.0,18.0,41.207573,28.0,FA
1544,Taj Gibson,MIN,2017-11-04,31.0,0.0,10.0,4.0,2.0,2.0,1.0,12.0,40.855354,24.2,FA
354,Donovan Mitchell,UTA,2017-11-03,32.0,3.0,3.0,0.0,2.0,2.0,0.0,25.0,40.520928,22.3,FA
758,Kyle Kuzma,LAL,2017-11-03,39.0,1.0,13.0,1.0,1.0,0.0,1.0,21.0,40.076594,25.3,FA
730,Will Barton,DEN,2017-11-04,34.0,2.0,8.0,5.0,1.0,0.0,1.0,21.0,40.038288,27.0,FA
865,Rondae Hollis-Jefferson,BKN,2017-11-03,37.0,0.0,5.0,4.0,2.0,0.0,2.0,21.0,37.017746,29.0,FA
967,Luc Mbah a Moute,HOU,2017-11-03,34.0,2.0,4.0,2.0,2.0,1.0,1.0,20.0,35.752433,20.3,FA


In [122]:
n

55.324564023247746

In [123]:
print(n)

55.3245640232


In [131]:
'{:.2f}'.format(n)

'55.32'

In [163]:
nbastats.get_game_log('Rudy Gay')

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Team,FANT,NBA_FANTASY_PTS
822,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700126,2017-11-03,SAS vs. CHA,W,...,2.0,0.0,2.0,3.0,20.0,20.0,2.0,THUNDERING CORNBREAD,36.901025,25.5
828,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700117,2017-11-02,SAS vs. GSW,L,...,0.0,1.0,3.0,2.0,9.0,-16.0,2.0,THUNDERING CORNBREAD,17.699250,25.5
823,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700092,2017-10-30,SAS @ BOS,L,...,0.0,1.0,0.0,2.0,14.0,2.0,1.0,THUNDERING CORNBREAD,20.554859,25.5
827,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700086,2017-10-29,SAS @ IND,L,...,2.0,0.0,2.0,1.0,10.0,-4.0,1.0,THUNDERING CORNBREAD,24.188852,25.5
829,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700071,2017-10-27,SAS @ ORL,L,...,0.0,4.0,2.0,2.0,2.0,-16.0,1.0,THUNDERING CORNBREAD,26.202660,25.5
821,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700059,2017-10-25,SAS @ MIA,W,...,2.0,0.0,3.0,0.0,22.0,16.0,1.0,THUNDERING CORNBREAD,34.643343,25.5
826,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700046,2017-10-23,SAS vs. TOR,W,...,1.0,1.0,2.0,3.0,11.0,2.0,1.0,THUNDERING CORNBREAD,20.992259,25.5
825,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700031,2017-10-21,SAS @ CHI,W,...,2.0,0.0,1.0,2.0,12.0,12.0,1.0,THUNDERING CORNBREAD,22.115829,25.5
824,22017,200752.0,Rudy Gay,1.610613e+09,SAS,San Antonio Spurs,0021700011,2017-10-18,SAS vs. MIN,W,...,1.0,0.0,0.0,3.0,14.0,3.0,1.0,THUNDERING CORNBREAD,25.978512,25.5
